In [1]:
import cv2 
import numpy as np
import os
from matplotlib import pyplot as plt 
import time
import mediapipe as mp



**keypoints using mp holistic**

In [2]:
mp_holistic=mp.solutions.holistic #the complete face drawing
mp_drawing=mp.solutions.drawing_utils # drawing on the face

In [3]:
def mediapipe_detection(image,model):
       ## could not edit the image
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)  #COLOR CODE CONVERSION FROM BGR TO RGB
    image.flags.writeable=False## could not edit the image
    results=model.process(image)  # result after prediction
    image.flags.writeable=True
    image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image,results


In [4]:
def draw_landmarks(image,results):
    mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_TESSELATION,
                              mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1),
                              mp_drawing.DrawingSpec(color=(80,256,121),thickness=1,circle_radius=1)) #DRAWS THE FACE CONNECTIONS
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80,22,10),thickness=2,circle_radius=4),
                              mp_drawing.DrawingSpec(color=(80,44,121),thickness=2,circle_radius=2)) #DRAWS THE POSE CONNECTIONS
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(121,22,76),thickness=2,circle_radius=4),
                              mp_drawing.DrawingSpec(color=(80,44,121),thickness=2,circle_radius=2) ) #DRAWS THE LEFTHAND CONNECTIONS
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=4),
                              mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2) ) #DRAWS THE RIGHT HAND CONNECTIONS


In [5]:
cap=cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic :
    while cap.isOpened():
        #read feed
        ret,frame=cap.read()

        #make detections
        image,results=mediapipe_detection(frame,holistic)
        print(results)

        #draw the connections and landmarks
        draw_landmarks(image,results)   

        #show the screen
        cv2.imshow('openCV Feed',image)

        #Break gracefully
        if cv2.waitKey(10) & 0xFF==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

# ## copy the code then 


  

In [6]:
len(results.pose_landmarks.landmark)

33

**extracrt keypoint values:**

In [7]:
pose=[]
for res in results.pose_landmarks.landmark:
    test=np.array([res.x,res.y,res.z,res.visibility])
    pose.append(test)
pose   

[array([ 0.48846507,  1.03317976, -0.25693211,  0.97943604]),
 array([ 0.48210374,  1.00105178, -0.25429279,  0.97895056]),
 array([ 0.48397774,  1.00058639, -0.25401944,  0.98100948]),
 array([ 0.4858551 ,  1.00007737, -0.25365674,  0.97595799]),
 array([ 0.47229743,  0.99922079, -0.27929425,  0.97913074]),
 array([ 0.46760955,  0.99887472, -0.27927452,  0.98261821]),
 array([ 0.46664029,  0.99829286, -0.27877462,  0.98003161]),
 array([ 0.46755448,  1.02325118, -0.14320438,  0.97321182]),
 array([ 0.44774294,  1.03310323, -0.23379362,  0.98586303]),
 array([ 0.4804194 ,  1.08650041, -0.20019388,  0.90458935]),
 array([ 0.47865558,  1.08643866, -0.23395863,  0.92754602]),
 array([ 0.51668125,  1.29020774, -0.08085726,  0.17211194]),
 array([ 0.37672496,  1.23981428, -0.22839911,  0.23616558]),
 array([ 0.54518098,  1.27373326, -0.10866582,  0.02724424]),
 array([ 0.37846893,  1.1044656 , -0.48723894,  0.05274947]),
 array([ 0.52046287,  1.1332984 , -0.06333186,  0.02631643]),
 array([

In [8]:
def extract_keypoints(results):  
    pose=np.array([[res.x,res.y,res.z,res.visibility]for res in results.pose_landmarks.landmark ]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face=np.array([[res.x,res.y,res.z]for res in results.face_landmarks.landmark ]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh=np.array([[res.x,res.y,res.z]for res in results.left_hand_landmarks.landmark ]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh=np.array([[res.x,res.y,res.z]for res in results.right_hand_landmarks.landmark ]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,face,lh,rh]) 

In [9]:
#calling above function
extract_keypoints(results).shape

(1662,)

# **setup of folders for array collection**

In [10]:
DATA_PATH=os.path.join('MP_DATA')
actions=np.array(['hello','thanks','ILoveYou'])
no_sequences=30
sequence_length=30

## above code is just the initialisation

In [11]:
## to create folders

for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH,action,str(sequence)))
        except:
            pass    

**collect keypoints values for training and testing**

In [12]:
cap=cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic :
   
   for action in actions:
      for sequence in range(no_sequences):
        for frame_num in range(sequence_length):
        #read feed
                ret,frame=cap.read()

                #make detections
                image,results=mediapipe_detection(frame,holistic)
                print(results)

                #draw the connections and landmarks
                draw_landmarks(image,results)   
                
                #new apply wait logic

                if frame_num==0:
                    cv2.putText(image,'STARTING COLLECTION',(120,200),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),4,cv2.LINE_AA)
                    cv2.putText(image,'collecting frames for {} Video number {}'.format(action,sequence),(120,200),
                    cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)

                    #show to screen 
                    cv2.imshow('openCV Feed',image)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image,'collecting frames for {} Video number {}'.format(action,sequence),(120,200),
                    cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)    

                    #show the screen
                    cv2.imshow('openCV Feed',image)

                keypoints=extract_keypoints(results)
                npy_path=os.path.join(DATA_PATH,action,str(sequence),str(frame_num))
                np.save(npy_path,keypoints)

                #Break gracefully
                if cv2.waitKey(10) & 0xFF==ord('q'):
                   break
   cap.release()
   cv2.destroyAllWindows()  

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

**# ###### preprocess data and create labels and features**

In [13]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
## here we are just importing required packages

label_map={label:num for num,label in enumerate(actions)}

label_map

{'hello': 0, 'thanks': 1, 'ILoveYou': 2}

In [14]:
sequences,labels=[],[]
for action in actions:
    for sequence in range(no_sequences):
        window=[]
        for frame_num in range(sequence_length):
            res=np.load(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])    

In [15]:
## forming categories
x=np.array(sequences)
x.shape
y=to_categorical(labels).astype(int)
y

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0,

In [16]:
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.05)

y_test.shape

(5, 3)

**```
!!! Build and train LSTM Network
```**

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense
from tensorflow.keras.callbacks import TensorBoard

In [18]:
log_dir=os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [19]:
model=Sequential()
model.add(LSTM(64,return_sequences=True,activation='relu',input_shape=(30,1662)))
model.add(LSTM(128,return_sequences=True,activation='relu'))
model.add(LSTM(64,return_sequences=False,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(actions.shape[0],activation='softmax'))

In [20]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])
model.fit(X_train,y_train,epochs=200,callbacks=[tb_callback] )

Epoch 1/200
3/3 [==============================] - 12s 211ms/step - loss: 1.4168 - categorical_accuracy: 0.3412
Epoch 2/200
3/3 [==============================] - 0s 134ms/step - loss: 1.4716 - categorical_accuracy: 0.4235
Epoch 3/200
3/3 [==============================] - 0s 142ms/step - loss: 1.1009 - categorical_accuracy: 0.4118
Epoch 4/200
3/3 [==============================] - 0s 131ms/step - loss: 1.0665 - categorical_accuracy: 0.4000
Epoch 5/200
3/3 [==============================] - 0s 143ms/step - loss: 1.0235 - categorical_accuracy: 0.5294
Epoch 6/200
3/3 [==============================] - 0s 135ms/step - loss: 0.9774 - categorical_accuracy: 0.5882
Epoch 7/200
3/3 [==============================] - 0s 127ms/step - loss: 1.3896 - categorical_accuracy: 0.4941
Epoch 8/200
3/3 [==============================] - 0s 130ms/step - loss: 1.0122 - categorical_accuracy: 0.4235
Epoch 9/200
3/3 [==============================] - 0s 134ms/step - loss: 1.0271 - categorical_accuracy: 0.5647


In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-trai

**make predictions**

In [22]:
res=model.predict(X_test)
actions[np.argmax(res[2])]

1/1 [==============================] - 1s 1s/step


'ILoveYou'

In [23]:
actions[np.argmax(y_test[3])]

'ILoveYou'

**save model weights**

In [24]:
model.save('action.h5')
model.load_weights('action.h5')

**Evaluation using confusion matrix and accuracy**

In [25]:
from sklearn.metrics import multilabel_confusion_matrix,accuracy_score
yhat=model.predict(X_test)
ytrue=np.argmax(y_test,axis=1).tolist()
yhat=np.argmax(yhat,axis=1).tolist()

1/1 [==============================] - 0s 48ms/step


In [26]:
multilabel_confusion_matrix(ytrue,yhat)

array([[[4, 0],
        [1, 0]],

       [[3, 1],
        [0, 1]],

       [[2, 0],
        [0, 3]]], dtype=int64)

In [27]:
accuracy_score(ytrue,yhat)

0.8

**############ TEST IN REAL TIME #############################**

In [28]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [32]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

TypeError: only size-1 arrays can be converted to Python scalars

<Figure size 1800x1800 with 0 Axes>

In [33]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [ ]:
cap.release()
cv2.destroyAllWindows()